In [1]:
%matplotlib inline


Sequence Models and Long-Short Term Memory Networks
===================================================

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py

At this point, we have seen various feed-forward networks. That is,
there is no state maintained by the network at all. This might not be
the behavior we want. Sequence models are central to NLP: they are
models where there is some sort of dependence through time between your
inputs. The classical example of a sequence model is the Hidden Markov
Model for part-of-speech tagging. Another example is the conditional
random field.

A recurrent neural network is a network that maintains some kind of
state. For example, its output could be used as part of the next input,
so that information can propogate along as the network passes over the
sequence. In the case of an LSTM, for each element in the sequence,
there is a corresponding *hidden state* $h_t$, which in principle
can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model,
part-of-speech tags, and a myriad of other things.


LSTM's in Pytorch
~~~~~~~~~~~~~~~~~

Before getting to the example, note a few things. Pytorch's LSTM expects
all of its inputs to be 3D tensors. The semantics of the axes of these
tensors is important. The first axis is the sequence itself, the second
indexes instances in the mini-batch, and the third indexes elements of
the input. We haven't discussed mini-batching, so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [2]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [20]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [21]:
# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print("out:", out, "\nhidden:", hidden, '\n')

out: tensor([[[-0.4785, -0.3109, -0.1566]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.4785, -0.3109, -0.1566]]], grad_fn=<StackBackward>), tensor([[[-1.7379, -0.7764, -0.2700]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.5050, -0.0986,  0.1477]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.5050, -0.0986,  0.1477]]], grad_fn=<StackBackward>), tensor([[[-0.9401, -0.1817,  0.1908]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.2325,  0.2996,  0.4022]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.2325,  0.2996,  0.4022]]], grad_fn=<StackBackward>), tensor([[[-0.5746,  0.4480,  0.5650]]], grad_fn=<StackBackward>)) 

out: tensor([[[-0.0279,  0.2914,  0.1493]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[-0.0279,  0.2914,  0.1493]]], grad_fn=<StackBackward>), tensor([[[-0.0859,  0.5432,  0.1916]]], grad_fn=<StackBackward>)) 

out: tensor([[[0.1621, 0.1617, 0.0159]]], grad_fn=<StackBackward>) 
hidden: (tensor([[[0.1621, 0.1617, 0.0159]]], grad_fn=<StackBackward>), tens

In [22]:
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(inputs)
print(out)
print(hidden)

tensor([[[ 0.2927, -0.2960, -0.5345]],

        [[ 1.7612, -1.0814, -1.1957]],

        [[ 0.2728, -1.9361, -1.2940]],

        [[-0.4160, -0.2229,  0.2163]],

        [[ 0.5239,  0.6090,  0.7696]]])
tensor([[[-0.2434,  0.3961,  0.5102]],

        [[-0.1092,  0.4521,  0.6861]],

        [[-0.0508,  0.6531,  0.6779]],

        [[ 0.0516,  0.4963,  0.3378]],

        [[ 0.2076,  0.2732,  0.0846]]], grad_fn=<StackBackward>)
(tensor([[[0.2076, 0.2732, 0.0846]]], grad_fn=<StackBackward>), tensor([[[0.5179, 0.7412, 0.1040]]], grad_fn=<StackBackward>))


Example: An LSTM for Part-of-Speech Tagging
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.


Prepare data:



In [23]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


Create the model:



In [24]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [30]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.7902, -1.1744, -1.4388],
        [-0.8913, -1.1091, -1.3471],
        [-0.8219, -1.0954, -1.4873],
        [-0.8637, -1.1071, -1.3948],
        [-0.8489, -1.1626, -1.3493]])
tensor([[-0.0626, -3.4730, -3.5194],
        [-5.6880, -0.0325, -3.5535],
        [-2.9017, -3.0503, -0.1079],
        [-0.0713, -5.1233, -2.7666],
        [-3.7204, -0.0445, -3.9485]])


Exercise: Augmenting the LSTM part-of-speech tagger with character-level features
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In the example above, each word had an embedding, which served as the
inputs to our sequence model. Let's augment the word embeddings with a
representation derived from the characters of the word. We expect that
this should help significantly, since character-level information like
affixes have a large bearing on part-of-speech. For example, words with
the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of
word $w$. Let $x_w$ be the word embedding as before. Then
the input to our sequence model is the concatenation of $x_w$ and
$c_w$. So if $x_w$ has dimension 5, and $c_w$
dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the
characters of a word, and let $c_w$ be the final hidden state of
this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.




In [50]:
char_to_ix = {}
for sentence in training_data:
    for word in sentence[0]:
        for i in word:
            if i not in char_to_ix.keys():
                char_to_ix[i]=len(char_to_ix)
            
char_to_ix

{'T': 0,
 'h': 1,
 'e': 2,
 'd': 3,
 'o': 4,
 'g': 5,
 'a': 6,
 't': 7,
 'p': 8,
 'l': 9,
 'E': 10,
 'v': 11,
 'r': 12,
 'y': 13,
 'b': 14,
 'k': 15}

In [55]:
def word_to_charid(word):
    tup = []
    for letter in word:
        tup.append(char_to_ix[letter])
        
    return torch.tensor(tup, dtype=torch.long)

In [56]:
word_to_charid('The')

tensor([0, 1, 2])

In [58]:
char_embd = nn.Embedding(len(char_to_ix),10)
exp = char_embd(word_to_charid('The'))

In [59]:
exp

tensor([[ 0.5687, -0.4798, -0.3498,  0.3211,  0.8500,  0.0051,  0.6672, -0.5692,
         -0.0385, -1.5256],
        [ 0.6652,  1.1653,  0.2134,  0.3863,  1.2854, -0.1999,  1.3419, -0.7575,
         -0.7325, -0.6029],
        [ 0.3530,  0.5275,  1.1979,  0.0636,  0.3645, -2.2024,  1.0670,  0.8081,
          0.7241,  0.9191]], grad_fn=<EmbeddingBackward>)

In [46]:
char_lstm = nn.LSTM(10, 10)

In [85]:
out, hidden = char_lstm(exp.view(len(exp),1,-1))
out,hidden[0]

(tensor([[[-0.0487, -0.0262, -0.0476,  0.2024, -0.2146,  0.0440,  0.0017,
            0.1622, -0.0500, -0.1084]],
 
         [[-0.1682,  0.0048, -0.0721,  0.2477, -0.2436, -0.2171,  0.1408,
            0.1826, -0.0187, -0.2535]],
 
         [[-0.0400, -0.0619, -0.1487,  0.1101, -0.1976, -0.2235,  0.2390,
            0.0536, -0.1614, -0.3146]]], grad_fn=<StackBackward>),
 tensor([[[-0.0400, -0.0619, -0.1487,  0.1101, -0.1976, -0.2235,  0.2390,
            0.0536, -0.1614, -0.3146]]], grad_fn=<StackBackward>))

In [87]:
sent = training_data[0][0]
sent

['The', 'dog', 'ate', 'the', 'apple']

In [93]:
char_ten = []
for word in sent:
    _,hidden = char_lstm(char_embd(word_to_charid(word)).view(len(word),1,-1))
    print(hidden[0])

tensor([[[-0.0400, -0.0619, -0.1487,  0.1101, -0.1976, -0.2235,  0.2390,
           0.0536, -0.1614, -0.3146]]], grad_fn=<StackBackward>)
tensor([[[ 0.0160,  0.0446, -0.1488, -0.2433, -0.1780, -0.2648,  0.2271,
           0.0321, -0.1851, -0.1593]]], grad_fn=<StackBackward>)
tensor([[[-0.0617, -0.0709, -0.2334, -0.1945, -0.1256, -0.1876,  0.3148,
          -0.1416,  0.1069, -0.0444]]], grad_fn=<StackBackward>)
tensor([[[-0.0903, -0.0421, -0.1940, -0.0220, -0.1727, -0.2428,  0.2860,
           0.0084, -0.0191, -0.2089]]], grad_fn=<StackBackward>)
tensor([[[-0.1958, -0.0469, -0.1374,  0.1330, -0.2946, -0.1278,  0.3522,
           0.1662,  0.0944,  0.0571]]], grad_fn=<StackBackward>)


In [105]:
char_ten = torch.zeros((len(sent), 10))
for i, word in enumerate(sent):
    _,hidden = char_lstm(char_embd(word_to_charid(word)).view(len(word),1,-1))
    char_ten[i] = hidden[0]

In [106]:
char_ten

tensor([[-0.0400, -0.0619, -0.1487,  0.1101, -0.1976, -0.2235,  0.2390,  0.0536,
         -0.1614, -0.3146],
        [ 0.0160,  0.0446, -0.1488, -0.2433, -0.1780, -0.2648,  0.2271,  0.0321,
         -0.1851, -0.1593],
        [-0.0617, -0.0709, -0.2334, -0.1945, -0.1256, -0.1876,  0.3148, -0.1416,
          0.1069, -0.0444],
        [-0.0903, -0.0421, -0.1940, -0.0220, -0.1727, -0.2428,  0.2860,  0.0084,
         -0.0191, -0.2089],
        [-0.1958, -0.0469, -0.1374,  0.1330, -0.2946, -0.1278,  0.3522,  0.1662,
          0.0944,  0.0571]], grad_fn=<CopySlices>)

In [100]:
test = torch.zeros((2,3))
test

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [102]:
test[0] = torch.tensor([2,3,1])
test

tensor([[2., 3., 1.],
        [0., 0., 0.]])

In [94]:
word_embeddings = nn.Embedding(len(word_to_ix), 10)

In [107]:
word_vec = word_embeddings(prepare_sequence(sent, word_to_ix))
word_vec

tensor([[-1.7832e+00,  6.4986e-01,  1.0964e-05, -9.4343e-01, -2.5621e+00,
         -9.0340e-01, -1.1404e+00,  3.0870e-01,  9.2925e-01, -1.1099e+00],
        [ 2.9609e-01,  9.7363e-01, -6.8629e-02,  4.6452e-01,  1.3876e+00,
         -4.5444e-01,  8.8294e-01, -9.1006e-01, -1.3159e+00, -4.0962e-01],
        [ 5.9175e-01,  1.1058e+00, -4.1602e-01, -1.2163e-01, -5.2085e-01,
         -2.8114e-01, -1.5574e+00,  1.3124e+00,  5.1202e-01,  1.4130e+00],
        [-6.4393e-01, -3.5775e+00,  1.8989e-01, -5.9293e-01, -8.3293e-01,
         -3.5120e-01,  5.8266e-01, -1.8223e+00,  2.7854e-01, -1.9936e+00],
        [-1.1515e+00,  6.8468e-01,  3.5912e-01,  6.1814e-01,  3.1925e-01,
         -4.7079e-01,  2.5327e-01, -1.7997e-01, -3.7535e-02,  3.3131e-01]],
       grad_fn=<EmbeddingBackward>)

In [108]:
char_ten.size(), word_vec.size()

(torch.Size([5, 10]), torch.Size([5, 10]))

In [115]:
torch.cat((word_vec, char_ten), dim=1).size()

torch.Size([10, 10])

In [152]:
class LSTMTagger_char(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, char_embedding_dim, char_hidden_dim
                 ,vocab_size, tagset_size, char_size ):
        super(LSTMTagger_char, self).__init__()
        self.hidden_dim = hidden_dim
        self.char_hidden_dim = char_hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.char_embeddings = nn.Embedding(char_size, char_embedding_dim)
        self.char_lstm = nn.LSTM(char_embedding_dim, char_hidden_dim)
    
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence, sentence_in):
        embeds = self.word_embeddings(sentence_in)
        
        char_ten = torch.zeros((len(sentence), self.char_hidden_dim))
        for i, word in enumerate(sentence):
            _,hidden = char_lstm(char_embd(word_to_charid(word)).view(len(word),1,-1))
            char_ten[i] = hidden[0]
        
        vec = torch.cat((embeds, char_ten),dim=1)
        
        lstm_out, _ = self.lstm(vec.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [154]:
model = LSTMTagger_char(EMBEDDING_DIM, HIDDEN_DIM,EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix), len(char_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(training_data[0][0], inputs)
#     print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence, sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
# with torch.no_grad():
#     inputs = prepare_sequence(training_data[0][0], word_to_ix)
#     tag_scores = model(training_data[0][0], inputs)

#     # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
#     # for word i. The predicted tag is the maximum scoring tag.
#     # Here, we can see the predicted sequence below is 0 1 2 0 1
#     # since 0 is index of the maximum value of row 1,
#     # 1 is the index of maximum value of row 2, etc.
#     # Which is DET NOUN VERB DET NOUN, the correct sequence!
#     print(tag_scores)

RuntimeError: The expanded size of the tensor (6) must match the existing size (10) at non-singleton dimension 0.  Target sizes: [6].  Tensor sizes: [10]

In [120]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\lih09\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [132]:
wsj = list(nltk.corpus.treebank.tagged_sents())

In [140]:
wsj[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [142]:
sent = []
tt = []
for word, vv in wsj[0]:
    sent.append(word)
    tt.append(vv)
sent, tt

(['Pierre',
  'Vinken',
  ',',
  '61',
  'years',
  'old',
  ',',
  'will',
  'join',
  'the',
  'board',
  'as',
  'a',
  'nonexecutive',
  'director',
  'Nov.',
  '29',
  '.'],
 ['NNP',
  'NNP',
  ',',
  'CD',
  'NNS',
  'JJ',
  ',',
  'MD',
  'VB',
  'DT',
  'NN',
  'IN',
  'DT',
  'JJ',
  'NN',
  'NNP',
  'CD',
  '.'])

In [143]:
training_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

In [146]:
data = []
for example in wsj:
    sent = []
    tt = []
    for word, vv in example:
        sent.append(word)
        tt.append(vv)
    data.append((sent,tt))

In [147]:
len(data)

3914

In [156]:

word_to_ix = {}
tag_to_ix = {}
for sent, tags in data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
            
#print(word_to_ix)
print(tag_to_ix)
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 64
HIDDEN_DIM = 64

{'NNP': 0, ',': 1, 'CD': 2, 'NNS': 3, 'JJ': 4, 'MD': 5, 'VB': 6, 'DT': 7, 'NN': 8, 'IN': 9, '.': 10, 'VBZ': 11, 'VBG': 12, 'CC': 13, 'VBD': 14, 'VBN': 15, '-NONE-': 16, 'RB': 17, 'TO': 18, 'PRP': 19, 'RBR': 20, 'WDT': 21, 'VBP': 22, 'RP': 23, 'PRP$': 24, 'JJS': 25, 'POS': 26, '``': 27, 'EX': 28, "''": 29, 'WP': 30, ':': 31, 'JJR': 32, 'WRB': 33, '$': 34, 'NNPS': 35, 'WP$': 36, '-LRB-': 37, '-RRB-': 38, 'PDT': 39, 'RBS': 40, 'FW': 41, 'UH': 42, 'SYM': 43, 'LS': 44, '#': 45}


In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-3.8764, -3.7409, -3.8481, -3.9418, -3.9173, -3.9522, -3.7120, -3.9219,
         -3.7228, -3.9821, -3.8979, -3.8709, -3.8561, -3.8063, -3.9160, -3.6379,
         -3.9320, -3.8169, -3.7379, -3.7923, -3.9456, -3.7972, -3.9303, -3.7655,
         -3.8521, -3.6153, -3.8334, -3.8298, -3.7773, -3.9701, -3.9679, -3.7633,
         -3.8201, -3.8374, -3.9446, -3.6983, -3.8390, -3.7510, -3.6550, -3.7869,
         -3.6299, -3.7952, -3.8491, -3.8478, -3.9322, -4.0355],
        [-3.8245, -3.8144, -3.7388, -3.9041, -3.8459, -3.9339, -3.7567, -3.9072,
         -3.7484, -3.8215, -3.9124, -3.8258, -3.8126, -3.8808, -4.0223, -3.5737,
         -3.8645, -3.7058, -3.8328, -3.8028, -3.9410, -3.7901, -3.9759, -3.8500,
         -3.8742, -3.7302, -3.7876, -3.8023, -3.7700, -3.8519, -4.0148, -3.7772,
         -3.7691, -3.8855, -3.9164, -3.6124, -3.9310, -3.7684, -3.6718, -3.8710,
         -3.6223, -3.8906, -3.9613, -3.8650, -3.9715, -3.9202],
        [-3.8504, -3.8841, -3.7788, -3.9211, -3.6939, -3.9064,

         -3.7997, -3.9197, -3.8640, -3.7612, -3.7434, -3.8587]])
